# feature_engineering

In [1]:
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

In [2]:
import nbimporter
from pre_processing import load_train

Importing Jupyter notebook from pre_processing.ipynb


### Funciones auxiliares

In [3]:
def corr(df):
    return df.corr()['precio'].sort_values(ascending=False)[1:]

In [4]:
def aniomes(anio, mes):
    if len(str(mes)) == 1:
        return int(str(anio)+'0'+str(mes))
    return int(str(anio)+str(mes))

In [19]:
def contiene_indicador_jardin(descripcion):
    indicadores = ["jardin","patio"]
    for indicador in indicadores:
        if indicador in str(descripcion).lower():
            return 1
    return 0

In [21]:
def contiene_vigilancia(descripcion):
    indicadores = ["vigilancia", "seguridad"]
    for indicador in indicadores:
        if indicador in str(descripcion).lower():
            return 1
    return 0

# Desarollo

In [5]:
df = load_train()

In [6]:
df.head(2)

,titulo,descripcion,direccion,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Apartamento,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,2015-08-23,0,0,0,0,0,2273000.0
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,Casa en condominio,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,2013-06-28,0,0,0,1,1,3600000.0


In [7]:
features = {}
features['originales'] = df.columns.values

### Variables categoricas. Encoding.

In [8]:
var_categoricas = ['tipodepropiedad', 'ciudad', 'provincia']

In [9]:
# label encoding
encoder = LabelEncoder()
encoded = df[var_categoricas].apply(encoder.fit_transform).add_suffix("_labelencoded")
encoded.to_csv('./features/categoricas_labelencoded.csv')
features['categoricas_labelencoded'] = encoded.columns.values

In [10]:
# onehot 

# para tipodepropiedad
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['tipodepropiedad']).add_suffix("_ohencoded")
encoded.to_csv('./features/tipodepropiedad_ohencoded.csv')
features['tipodepropiedad_ohencoded'] = encoded.columns.values

# para provincias
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['provincia']).add_suffix("_ohencoded")
encoded.to_csv('./features/provincia_ohencoded.csv')
features['provincia_ohencoded'] = encoded.columns.values

### Generacion de nuevas features

In [11]:
df.columns

Index(['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'gimnasio',
       'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

##### FECHA

In [12]:
df_fecha = df[['fecha', 'precio', 'metroscubiertos']].copy()

df_fecha['anio'] = df_fecha['fecha'].dt.year
df_fecha['mes'] = df_fecha['fecha'].dt.month
df_fecha['dia'] = df_fecha['fecha'].dt.day
df_fecha['timestamp'] = df_fecha['fecha'].apply(lambda x: datetime.timestamp(x))
df_fecha['aniomes'] = df_fecha.apply(lambda row: aniomes(row['anio'], row['mes']), axis=1)

In [13]:
# calculamos el precio promedio del metro cubierto por aniomes
precio_mtcubierto_aniomes = df_fecha.groupby(['aniomes'])['metroscubiertos', 'precio'].agg(sum).apply(lambda x: x['precio']/x['metroscubiertos'], axis=1)
df_fecha['precio_promedio_metrocubierto_aniomes'] = df_fecha['aniomes'].apply(lambda x: precio_mtcubierto_aniomes[x])

In [14]:
# calculamos el precio promedio del metro cubierto por mes
precio_mtcubierto_mes = df_fecha.groupby(['mes'])['metroscubiertos', 'precio'].agg(sum).apply(lambda x: x['precio']/x['metroscubiertos'], axis=1)
df_fecha['precio_promedio_metrocubierto_mes'] = df_fecha['mes'].apply(lambda x: precio_mtcubierto_mes[x])

In [15]:
df_fecha = df_fecha.drop(['precio', 'metroscubiertos'], axis=1)
pasar_a_enteros = ['precio_promedio_metrocubierto_mes', 'precio_promedio_metrocubierto_aniomes']
df_fecha[pasar_a_enteros] = df_fecha[pasar_a_enteros].astype('uint16')

In [16]:
df_fecha.to_csv('./features/fecha.csv')
features['fecha'] = df_fecha.columns.values

In [17]:
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_fecha['aniomes'].astype('category')).add_suffix("_ohencoded")
encoded.to_csv('./features/aniomes_ohencoded.csv')
features['aniomes_ohencoded'] = encoded.columns.values

##### DESCRIPCION

In [18]:
df.columns

Index(['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'gimnasio',
       'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [31]:
df_descripcion = df[['descripcion']].copy()

df_descripcion["jardin"] = df_descripcion["descripcion"].map(contiene_indicador_jardin)
df_descripcion["vigilancia"] = df_descripcion["descripcion"].map(contiene_vigilancia)

In [32]:
df_descripcion.head(2)

,descripcion,jardin,vigilancia
id,,,
254099,"depto. interior de 80.15m2, consta de sala com...",0,0
53461,"<p>entre sonora y guerrero, atr&aacute;s del h...",0,0


In [36]:
df_descripcion.drop('descripcion', inplace=True, axis=1)
df_descripcion.to_csv('./features/descripcion.csv')
features['descripcion'] = df_descripcion.columns.values